In [79]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, roc_auc_score, make_scorer

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

from tqdm import tqdm
from functools import reduce
from copy import deepcopy
import gc
from collections import Counter

%matplotlib inline

import warnings
warnings.filterwarnings('ignore') 

# %cd ../

In [80]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/X_prepared.csv     
  inflating: data/y_prepared.csv     
  inflating: data/test_df_prepared.csv  
  inflating: data/test_ids.csv       


In [45]:
def delete_double_spaces(nm):
    new_nm = ''
    for char in nm:
        if char != ' ' or (len(new_nm) and new_nm[-1] != ' '):
            new_nm += char
    return new_nm.strip()


def label_process(nm):
    nm = nm.lower()
    new_nm = ''
    balance = 0
    for char in nm:
        if char in '({[':
            balance += 1
        elif char in ')}]':
            balance = max(0, balance - 1)
        elif balance == 0:
            new_nm += char
    nm = new_nm
    nm = delete_double_spaces(nm)
    return nm

In [46]:
df = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")
test_ids = deepcopy(test_data.id)

FileNotFoundError: [Errno 2] File b'data/train.csv' does not exist: b'data/train.csv'

In [ ]:
df['processed_name'] = df.name_mfi.apply(label_process)
test_data['processed_name'] = test_data.name_mfi.apply(label_process)

In [ ]:
df['item_hash'] = df['name_mfi'].apply(hash) ^ df.weight_mfi.apply(lambda x: hash(str(x))) ^ df.weight.apply(lambda x: hash(str(x))) ^ \
    df.mailtype.apply(lambda x: hash(str(x))) ^ df.transport_pay.apply(lambda x: hash(str(x)))

test_data['item_hash'] = test_data['item_hash'] = test_data['name_mfi'].apply(hash) ^ test_data.weight_mfi.apply(lambda x: hash(str(x))) ^ test_data.weight.apply(lambda x: hash(str(x))) ^ \
    test_data.mailtype.apply(lambda x: hash(str(x))) ^ test_data.transport_pay.apply(lambda x: hash(str(x)))

In [ ]:
hash_counter = Counter(pd.concat([df.item_hash, test_data.item_hash], ignore_index=True))
name_counter = Counter(pd.concat([df.processed_name, test_data.processed_name], ignore_index=True))

In [ ]:
id_hash = pd.concat([df[['id', 'item_hash']], test_data[['id', 'item_hash']]], ignore_index=True)
id_hash = id_hash.groupby('item_hash').agg(max).rename(columns={'id': 'last_id'})

In [ ]:
df = df.join(id_hash, on='item_hash')
test_data = test_data.join(id_hash, on='item_hash')

In [ ]:
X = df.drop(columns=["label"])
y = df.label
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
top_operators = [obj for obj, count in Counter(test_data['index_oper'].replace(' ', '0').astype(float).astype(int)).most_common()][:10]

In [ ]:
df.item_hash.nunique()

In [ ]:
useless = []
cat_features = ["type", "is_privatecategory", "class", "is_in_yandex", "mailctg", "directctg", "mailtype"]
cat_features = list(set(cat_features) - set(useless))

def prepare_data(df):
    oper_type, oper_attr = zip(*df['oper_type + oper_attr'].map(lambda x: x.split('_')))
    
    # df['is_negative_oper_attr'] = pd.Series(oper_attr) == '-1'
    df['specialization_ratio'] = df.total_qty_over_index_and_type / df.total_qty_over_index.clip(lower=1)
    df['total_over_person'] = df.total_qty_over_index / df.dist_qty_oper_login_1.clip(lower=1)
    df['unknown_oper_ratio'] = df.total_qty_oper_login_0 / df.total_qty_oper_login_1.clip(lower=1)
    
    df['same_hash'] = df.item_hash.apply(lambda h: hash_counter[h])
    df['same_name'] = df.processed_name.apply(lambda name: name_counter[name])
    
    df['isnt_last'] = (df.id == df.last_id) * (df.same_hash < 30)
    
    for operator in top_operators:
        df[f'is_{operator}_operator'] = df['index_oper'].replace(' ', '0').astype(float).astype(int) == operator
    
    replace_zeros = ['priority', 'is_privatecategory', 'is_in_yandex', 'mailtype', 'directctg']
    for column in replace_zeros:
        df[column][df[column] == 0] = df[column].mode()[0]
        df[column][df[column] == '0'] = df[column].mode()[0]
    
    df.priority = 1 * (df.priority == 7503.) + 2 * (df.priority == 7504.) + 3 * (df.priority == 7506.)
    
    df.index_oper = df.index_oper.replace(' ', '0').astype(float).astype(int)
    
    for cat_feature in cat_features:
        df[cat_feature] = df[cat_feature].apply(lambda text: str(text))
        
    df['name_mfi_count'] = df.name_mfi.apply(lambda name: 0 if name == '0' else len(name.split(',')))
    
    df.is_return = df.is_return.apply(lambda text: text == 'Y')
        
    df.drop(columns=["oper_type + oper_attr"], inplace=True)
    df.drop(columns=["name_mfi", "item_hash", "processed_name"], inplace=True)
    df.drop(columns=['mailrank'], inplace=True)
    df.drop(columns=useless, inplace=True, errors='ignore')
    
    return df

In [ ]:
X_train = prepare_data(X_train)
X_val = prepare_data(X_val)
test_df = prepare_data(test_data)

In [ ]:
X_train.to_csv('data/X_train_prepared.csv', index=False)
X_val.to_csv('data/X_val_prepared.csv', index=False)
y_train.to_csv('data/y_train_prepared.csv', index=False)
y_val.to_csv('data/y_val_prepared.csv', index=False)
test_df.to_csv('data/test_df_prepared.csv', index=False)
y_train.to_csv('data/y_train_prepared.csv', index=False)
y_val.to_csv('data/y_val_prepared.csv', index=False)

In [81]:
useless = []
cat_features = ["type", "is_privatecategory", "class", "is_in_yandex", "mailctg", "directctg", "mailtype"]
cat_features = list(set(cat_features) - set(useless))

cat_features = list(set(cat_features) - set(useless))
def to_cat_features(df):
    for cat_feature in cat_features:
        df[cat_feature] = df[cat_feature].apply(lambda text: str(text))
        
# X_train = pd.read_csv('data/X_train_prepared.csv')
# y_train = pd.read_csv('data/y_train_prepared.csv')
# X_val = pd.read_csv('data/X_val_prepared.csv')
# y_val = pd.read_csv('data/y_val_prepared.csv')
# test_df = pd.read_csv('data/test_df_prepared.csv')
# test_ids = pd.read_csv('data/test_ids.csv')

X_train = pd.read_csv('data/X_prepared.csv') 
y_train = pd.read_csv('data/y_prepared.csv') 
test_df = pd.read_csv('data/test_df_prepared.csv')
test_ids = pd.read_csv('data/test_ids.csv')


to_cat_features(X_train)
# to_cat_features(X_val)
to_cat_features(test_df)

In [82]:
model = CatBoostClassifier(random_state=0, max_depth=8, verbose=50, iterations=7000, auto_class_weights="Balanced", cat_features=cat_features)

In [83]:
#!c1.32
model.fit(X_train, y_train)

Learning rate set to 0.070999
0:	learn: 0.6087893	total: 1.87s	remaining: 3h 38m 25s
50:	learn: 0.2403778	total: 55.5s	remaining: 2h 6m 3s
100:	learn: 0.2309639	total: 1m 45s	remaining: 2h 36s
150:	learn: 0.2268286	total: 2m 35s	remaining: 1h 57m 55s
200:	learn: 0.2244060	total: 3m 27s	remaining: 1h 56m 53s
250:	learn: 0.2222753	total: 4m 19s	remaining: 1h 56m 15s
300:	learn: 0.2204358	total: 5m 8s	remaining: 1h 54m 36s
350:	learn: 0.2188437	total: 5m 57s	remaining: 1h 53m 1s
400:	learn: 0.2173278	total: 6m 46s	remaining: 1h 51m 33s
450:	learn: 0.2162380	total: 7m 37s	remaining: 1h 50m 44s
500:	learn: 0.2151597	total: 8m 30s	remaining: 1h 50m 18s
550:	learn: 0.2142220	total: 9m 19s	remaining: 1h 49m 10s
600:	learn: 0.2132864	total: 10m 11s	remaining: 1h 48m 26s
650:	learn: 0.2123196	total: 11m 2s	remaining: 1h 47m 39s
700:	learn: 0.2113327	total: 11m 52s	remaining: 1h 46m 40s
750:	learn: 0.2103901	total: 12m 42s	remaining: 1h 45m 45s
800:	learn: 0.2095947	total: 13m 32s	remaining: 1h 4

In [84]:
def score_model(model, X, y, threshold=0.5):  # 0.225 is the best for now
    pred = model.predict_proba(X)[:, 1] > threshold
    recall = recall_score(y, pred, average="macro")
    roc_auc = roc_auc_score(y,  pred, multi_class='ovo')
    score = 0.1 * recall + 0.9 * roc_auc
    return score

In [74]:
score_model(model, X_val, y_val, 0.35)

0.9001320410221255

In [85]:
score_model(model, X_train, y_train, 0.35) 

0.9319976456659861

In [86]:
sorted(list(zip(X_train.columns, model.feature_importances_)), key = lambda x: -x[1])

[('total_over_person', 8.617996978373908),
 ('same_oper_item_hash', 7.8616569382495305),
 ('unknown_oper_ratio', 6.814557509207496),
 ('total_qty_oper_login_1', 6.562439660358021),
 ('specialization_ratio', 6.547828256602237),
 ('index_oper', 6.478515541710767),
 ('total_qty_oper_login_0', 6.217059156277348),
 ('same_hash', 4.785745907603959),
 ('mailctg', 4.740562340610744),
 ('transport_pay', 4.107319388997391),
 ('weight', 3.7353274808759487),
 ('total_qty_over_index_and_type', 3.5007545139541696),
 ('id', 3.2922648553623026),
 ('type', 3.220401970452459),
 ('weight_mfi', 2.777887092850324),
 ('price_mfi', 2.634764016710684),
 ('dist_qty_oper_login_1', 2.3899338299096664),
 ('same_name', 2.321428676916351),
 ('last_id', 2.299719693096319),
 ('total_qty_over_index', 2.160941647602158),
 ('name_mfi_len', 2.103709257870011),
 ('class', 1.5188161815871413),
 ('priority', 1.4128383450112492),
 ('name_mfi_count', 1.1585250088888095),
 ('is_wrong_phone_number', 0.7636340496122475),
 ('is_w

In [87]:
test_preds = (model.predict_proba(test_df)[:, 1])
submission = pd.DataFrame({'id': test_ids.id, 'label': test_preds})
submission.to_csv("submission.csv", index=False)

In [88]:
4

4

In [ ]:
unrounded_sub = pd.read_csv('submission.csv') 
sub = unrounded_sub.copy()
sub.label = (unrounded_sub.label > 0.225).astype(int) 
sub.to_csv('submission_rounded.csv', index=False)

In [ ]:
sub.label.mean()
unrounded_sub.label.mean()